# 🚀 Test GPU - RTX 5070 avec CUDA 12.8

Ce notebook teste la configuration GPU avec TensorFlow et PyTorch.

## 1. Vérification de l'environnement

In [ ]:
import sys
import platform

print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")

## 2. Test TensorFlow GPU

In [ ]:
import tensorflow as tf
import time

print(f"TensorFlow version: {tf.__version__}")
print(f"Built with CUDA: {tf.test.is_built_with_cuda()}")
print(f"\nGPUs disponibles: {len(tf.config.list_physical_devices('GPU'))}")

gpus = tf.config.list_physical_devices('GPU')
for i, gpu in enumerate(gpus):
    print(f"  GPU {i}: {gpu.name}")
    details = tf.config.experimental.get_device_details(gpu)
    print(f"    Compute Capability: {details.get('compute_capability', 'N/A')}")

In [ ]:
# Benchmark TensorFlow GPU vs CPU
print("Benchmark: Multiplication matricielle 5000x5000")
print("-" * 50)

# Test GPU
if gpus:
    with tf.device('/GPU:0'):
        a = tf.random.normal([5000, 5000])
        b = tf.random.normal([5000, 5000])
        
        start = time.time()
        c = tf.matmul(a, b)
        _ = c.numpy()  # Force l'exécution
        gpu_time = time.time() - start
    print(f"⚡ GPU: {gpu_time:.4f} secondes")

# Test CPU
with tf.device('/CPU:0'):
    a = tf.random.normal([5000, 5000])
    b = tf.random.normal([5000, 5000])
    
    start = time.time()
    c = tf.matmul(a, b)
    _ = c.numpy()
    cpu_time = time.time() - start
print(f"🐢 CPU: {cpu_time:.4f} secondes")

if gpus:
    speedup = cpu_time / gpu_time
    print(f"\n📊 Accélération GPU: {speedup:.2f}x plus rapide!")

## 3. Test PyTorch GPU

In [ ]:
import torch

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"cuDNN version: {torch.backends.cudnn.version()}")
    print(f"\nGPUs détectés: {torch.cuda.device_count()}")
    
    for i in range(torch.cuda.device_count()):
        props = torch.cuda.get_device_properties(i)
        print(f"\n  GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"    Compute Capability: {props.major}.{props.minor}")
        print(f"    Total Memory: {props.total_memory / 1024**3:.2f} GB")
        print(f"    Multi-Processors: {props.multi_processor_count}")

In [ ]:
# Benchmark PyTorch GPU vs CPU
print("Benchmark: Multiplication matricielle 5000x5000")
print("-" * 50)

if torch.cuda.is_available():
    # Test GPU
    device = torch.device('cuda')
    a = torch.randn(5000, 5000, device=device)
    b = torch.randn(5000, 5000, device=device)
    
    # Warm-up
    _ = torch.matmul(a, b)
    torch.cuda.synchronize()
    
    start = time.time()
    c = torch.matmul(a, b)
    torch.cuda.synchronize()
    gpu_time = time.time() - start
    print(f"⚡ GPU: {gpu_time:.4f} secondes")
    
    # Mémoire GPU
    print(f"\n💾 Mémoire GPU allouée: {torch.cuda.memory_allocated(0) / 1024**2:.2f} MB")
    print(f"Mémoire GPU réservée: {torch.cuda.memory_reserved(0) / 1024**2:.2f} MB")

# Test CPU
device = torch.device('cpu')
a = torch.randn(5000, 5000, device=device)
b = torch.randn(5000, 5000, device=device)

start = time.time()
c = torch.matmul(a, b)
cpu_time = time.time() - start
print(f"🐢 CPU: {cpu_time:.4f} secondes")

if torch.cuda.is_available():
    speedup = cpu_time / gpu_time
    print(f"\n📊 Accélération GPU: {speedup:.2f}x plus rapide!")

## 4. Test d'entraînement simple (TensorFlow)

In [ ]:
# Créer un modèle simple CNN
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print("Modèle créé:")
model.summary()

In [ ]:
# Créer des données synthétiques
import numpy as np

x_train = np.random.random((1000, 28, 28, 1)).astype('float32')
y_train = np.random.randint(10, size=(1000,))

print("Entraînement du modèle sur GPU...")
history = model.fit(
    x_train, y_train,
    batch_size=32,
    epochs=3,
    verbose=1
)

print("\n✅ Entraînement terminé sur GPU!")

## 5. Test d'entraînement simple (PyTorch)

In [ ]:
import torch.nn as nn
import torch.optim as optim

# Définir un modèle simple
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(64 * 5 * 5, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 64 * 5 * 5)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleCNN().to(device)

print(f"Modèle créé et déplacé sur: {device}")
print(f"Nombre de paramètres: {sum(p.numel() for p in model.parameters())}")

In [ ]:
# Créer des données synthétiques
x_train = torch.randn(100, 1, 28, 28).to(device)
y_train = torch.randint(0, 10, (100,)).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Entraînement du modèle sur GPU...")
model.train()

for epoch in range(3):
    optimizer.zero_grad()
    outputs = model(x_train)
    loss = criterion(outputs, y_train)
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch+1}/3 - Loss: {loss.item():.4f}")

print("\n✅ Entraînement terminé sur GPU!")

## 6. Résumé

Si toutes les cellules ci-dessus se sont exécutées sans erreur, votre environnement GPU est correctement configuré! 🎉

### Points de vérification:
- ✅ TensorFlow détecte le GPU
- ✅ PyTorch détecte CUDA
- ✅ Accélération GPU significative vs CPU
- ✅ Entraînement de modèles fonctionnel sur GPU

### Prochaines étapes:
- Commencer vos projets de Deep Learning
- Expérimenter avec des datasets réels
- Utiliser TensorBoard pour le monitoring
- Explorer les optimisations GPU avancées